# Test Tiny A11y LoRA Model
This notebook loads your LoRA fine-tuned `younglim/tiny-a11y-model` on DeepSeek-Coder and runs prompts.

In [ ]:
# Install required packages
!pip install --upgrade pip
!pip install transformers accelerate peft

In [ ]:
# Load base model and tokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from peft import PeftModel

base_model_name = 'deepseek-ai/deepseek-coder-1.3b-instruct'
lora_model_name = 'younglim/tiny-a11y-model'

tokenizer = AutoTokenizer.from_pretrained(base_model_name, use_fast=True)
base_model = AutoModelForCausalLM.from_pretrained(base_model_name, device_map='auto')

# Apply LoRA weights
model = PeftModel.from_pretrained(base_model, lora_model_name, device_map='auto')

In [ ]:
# Create text-generation pipeline
generator = pipeline('text-generation', model=model, tokenizer=tokenizer, device=0)

# Example prompt
prompt = 'Explain why alt text is important for images.'
output = generator(prompt, max_new_tokens=100)
print(output[0]['generated_text'])

In [ ]:
# Interactive prompt testing
custom_prompt = input('Enter your prompt: ')
result = generator(custom_prompt, max_new_tokens=100)
print(result[0]['generated_text'])